<a href="https://colab.research.google.com/github/dardeshna/frc-stuffs/blob/master/linalg/opr_2022cabl_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Offensive Power Rating (OPR)

Offensive Power Rating (OPR) is a metric to estimate a team's average point contribution to their alliance. We can calculate OPR using the match scores from an event.

In [ ]:
# setup
import numpy as np
import pandas as pd
import requests
import json

## Pulling data from TBA

The first step is to pull event data from [The Blue Alliance](https://www.thebluealliance.com/). We can use their Read API which is documented here: https://www.thebluealliance.com/apidocs/v3

To calculate OPR we will need a list of teams and scores for each match. We can also pull OPR from TBA for comparison.

In [ ]:
# generate a TBA auth key at https://www.thebluealliance.com/account
auth_key = ''

event = '2022cabl' # beach blitz 2022

# get a list of teams

# get match data

# get opr calculated by TBA


## Processing the data

Next we need to process the responses obtained by the TBA API. We can convert them to nested python lists and dictionaries using the `.json()` function. OPR is traditionally calculated using only quals matches, so we also want to filter by match type.

In [ ]:
# get a list of team names

# get a list of dictionaries with data for each match

# print the first 10 teams

# print qm1 data


## Calculating OPR
OPR just solves a system of equations to estimate each team's average point contribution. Specifically, for match $i$ we have the following equations:

\begin{gather*}
x_{{r_i}_a}+x_{{r_i}_b}+x_{{r_i}_c}=s_{r_i} \\
x_{{b_i}_a}+x_{{b_i}_b}+x_{{b_i}_c}=s_{b_i} \\
\end{gather*}

Here, $x_{{r_i}_a}$ represents the OPR of the first team on the red alliance in match $i$. Similarly, $s_{r_i}$ is the red alliance's score in match $i$.

To be clear, these symbols are just placeholders. If teams 8, 254, and 1678 scored 133 points for the red alliance while 971, 973, and 1323 scored 128 points for blue, we would have:

\begin{gather*}
x_{8}+x_{254}+x_{1678}=133 \\
x_{971}+x_{973}+x_{1323}=128
\end{gather*}

For $m$ matches, we can stack the two equations from each match to get a total of $2m$ equations:

\begin{gather*}
x_{{r_1}_a}+x_{{r_1}_b}+x_{{r_1}_c}=s_{r_1} \\
x_{{b_1}_a}+x_{{b_1}_b}+x_{{b_1}_c}=s_{b_1} \\
x_{{r_2}_a}+x_{{r_2}_b}+x_{{r_2}_c}=s_{r_2} \\
x_{{b_2}_a}+x_{{b_2}_b}+x_{{b_2}_c}=s_{b_2} \\
\vdots \\
x_{{r_m}_a}+x_{{r_m}_b}+x_{{r_m}_c}=s_{r_m} \\
x_{{b_m}_a}+x_{{b_m}_b}+x_{{b_m}_c}=s_{b_m} \\
\end{gather*}

These equations can be written in matrix form:

\begin{gather*}
\begin{bmatrix}
x_{{r_1}_a}+x_{{r_1}_b}+x_{{r_1}_c} \\
x_{{b_1}_a}+x_{{b_1}_b}+x_{{b_1}_c} \\
x_{{r_2}_a}+x_{{r_2}_b}+x_{{r_2}_c} \\
x_{{b_2}_a}+x_{{b_2}_b}+x_{{b_2}_c} \\
\vdots \\
x_{{r_m}_a}+x_{{r_m}_b}+x_{{r_m}_c} \\
x_{{b_m}_a}+x_{{b_m}_b}+x_{{b_m}_c}
\end{bmatrix}
= \begin{bmatrix} s_{r_1} \\ s_{b_1} \\ s_{r_2} \\ s_{b_2} \\ \vdots \\ s_{r_m} \\ s_{b_m} \end{bmatrix}
\end{gather*}

If we have $n$ teams at the event, we can break up the left-hand side into the product of $\mathbf{A} \in \mathbb{R}^{2m \times n}$ and $\mathbf{x} \in \mathbb{R}^{n}$. The entries of $\mathbf{x}$ contains the OPRs for each team, while each row of $\mathbf{A}$ is zero except for the columns corresponding to the teams on that alliance. After doing so, we end up with something like this:

\begin{gather*}
\mathbf{Ax} =
\begin{bmatrix}
1 & 0 & 0 & 1 & 0 & 1 & \dots & 0 \\
0 & 1 & 1 & 0 & 1 & 1 & \dots & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & \dots & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & \dots & 0 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & 0 & 0 & 0 & \dots & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & \dots & 0
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\ x_6 \\ \vdots \\x_n \\
\end{bmatrix}
= \begin{bmatrix} s_{r_1} \\ s_{b_1} \\ s_{r_2} \\ s_{b_2} \\ \vdots \\ s_{r_m} \\ s_{b_m} \end{bmatrix}
= \mathbf{b}
\end{gather*}

At a typical FRC event, $2m > n$ so we have more equations than unknowns and $\mathbf{A}$ is a tall matrix. This means that while there is no exact solution for $\mathbf{x}$, we can use `np.linalg.lstsq(A, b)` to solve for an $\mathbf{x}$ that minimizes the error $\| \mathbf{Ax}-\mathbf{b} \|$.


In [ ]:
# initialize matrices

# iterate over matches

    # fill in equation for red alliance (row 2*i)
    
    # fill in equation for blue alliance (row 2*i+1)

# solve for OPRs

## Comparison to TBA OPR

Finally, we can sort teams by OPR and compare our values to TBA's values!

In [ ]:
# get dictionary of OPRs from TBA

# build a new dictionary with both our OPR and TBA's OPR

# convert dict to a pandas dataframe and sort by our OPR